<a href="https://colab.research.google.com/github/tlacaelel666/Quasys/blob/main/QuantumNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install pylatexenc

In [ ]:
! pip install qiskit qiskit-aer

In [ ]:
import sys
import os
! pip  install numpy
! pip  install mpmath
! pip  install scikit-learn
! pip  install matplotlib
print("Instalación completada.")

In [15]:
from matplotlib import pyplot as plt
df_0['Counts'].plot(kind='hist', bins=20, title='Counts')
plt.gca().spines[['top', 'right',]].set_visible(False)

NameError: name 'df_0' is not defined

In [ ]:
# 1. Imports
import numpy as np
import math
from sklearn.decomposition import PCA
import pandas as pd

# 2. Configuración Global
# Amplitudes
a_ft = 0.8                    # Amplitud fase en el tiempo
a_mf = 0.9                    # Amplitud modulada por la fase
a_fn = complex(0, 1)          # Amplitud fase normalizada

# Pesos de contexto para el estado-vector
peso_a = 0.1
peso_b = 0.3
peso_c = 0.5
peso_d = 0.7

# Parámetros del sistema
s = 0.5                       # Incertidumbre (dagger)
t = 0.6                       # Tiempo para la evolución de la función de onda
n = 0.9                       # Porcentaje de neuronas objetivo
f = 0.001                     # Frecuencia mínima

# 3. Clase Base: SistemaNeuroQuantico
class SistemaNeuroQuantico:
    def __init__(self):
        self.estado_coherente = True
        self.estado_vector = None
        self.historico_colapsos = []

    def crear_superposicion(self, s_fti, s_fni, s_mfi):
        norm_fti = np.array(s_fti) / np.linalg.norm(s_fti) if s_fti else np.array([0])
        norm_fni = np.array(s_fni) / np.linalg.norm(s_fni) if s_fni else np.array([0])
        norm_mfi = np.array(s_mfi) / np.linalg.norm(s_mfi) if s_mfi else np.array([0])
        estado_base = np.array([
            a_ft * np.sum(norm_fti),
            a_fn * np.sum(norm_fni),
            a_mf * np.sum(norm_mfi)
        ], dtype=complex)
        return estado_base

    def funcion_onda(self, estado, tiempo):
        energia = np.array([f * i for i in range(1, len(estado) + 1)])
        fase = np.exp(-1j * energia * tiempo)
        return estado * fase

    def calcular_w_cuantico(self, s_fti, s_fni, s_mfi, a, b, c, d):
        estado_super = self.crear_superposicion(s_fti, s_fni, s_mfi)
        influencia = np.array([a*peso_a, b*peso_b, c*peso_c, d*peso_d])
        psi_t = self.funcion_onda(estado_super, t)
        estado_vector = np.zeros(4, dtype=complex)
        estado_vector[:3] = psi_t
        estado_vector[3] = np.sum(influencia) * s
        self.estado_vector = estado_vector / np.linalg.norm(estado_vector)
        return self.estado_vector

    def colapso_cuantico(self, umbral_decoherencia=0.7):
        if self.estado_vector is None:
            return None
        probs = np.abs(self.estado_vector)**2
        entropia = -np.sum(probs * np.log(probs + 1e-10))
        if entropia > umbral_decoherencia:
            self.estado_coherente = False
            idx = np.random.choice(len(probs), p=probs)
            vec_col = np.zeros_like(self.estado_vector)
            vec_col[idx] = 1.0
            fase_res = np.angle(self.estado_vector)
            resultado = {
                'vector_estado': self.estado_vector.copy(),
                'vector_colapsado': vec_col,
                'estado_medido': idx,
                'fase_residual': fase_res,
                'probabilidades': probs,
                'entropia': entropia
            }
            self.historico_colapsos.append(resultado)
            return resultado
        return {'vector_estado': self.estado_vector, 'coherente': True, 'entropia': entropia}

    def interpretar_estado_vector(self, res):
        if res.get('coherente', False):
            return {
                'tipo': 'superposicion',
                'amplitudes': np.abs(res['vector_estado']),
                'fases': np.angle(res['vector_estado']),
                'interpretacion': 'Estado en superposición coherente'
            }
        vec = res['vector_estado']
        real = np.real(vec); imag = np.imag(vec)
        return {
            'tipo': 'colapsado',
            'estado_clasico': res['estado_medido'],
            'vector_completo': {
                'amplitudes': np.abs(vec),
                'fases': res['fase_residual'],
                'componente_real': real,
                'componente_imaginaria': imag
            },
            'confianza': np.max(res['probabilidades']),
            'incertidumbre_residual': res['entropia'],
            'interpretacion': self._generar_interpretacion_textual(res)
        }

    def _generar_interpretacion_textual(self, r):
        estado = r['estado_medido']
        prob = r['probabilidades'][estado]
        return (f"Estado dominante: {estado} (p={prob:.3f}) | "
                f"Entropía: {r['entropia']:.3f} | "
                f"Fases residuales: {r['fase_residual']}")

# Auxiliar: Métricas Cuánticas básicas
def calcular_metricas_cuanticas(sistema, s_fti, s_fni, s_mfi, a=1, b=0.5, c=0.7, d=0.3):
    estado = sistema.calcular_w_cuantico(s_fti, s_fni, s_mfi, a, b, c, d)
    col = sistema.colapso_cuantico()
    interp = sistema.interpretar_estado_vector(col)
    return {'estado_cuantico': estado, 'colapso': col, 'interpretacion': interp}

# 4. Red Avanzada: RedNeuronalCuantica
class RedNeuronalCuantica(SistemaNeuroQuantico):
    def __init__(self, num_qubits=4, num_capas=3):
        super().__init__()
        self.num_qubits = num_qubits
        self.num_capas = num_capas
        self.matriz_densidad = None

    def crear_estado_entrelazado(self, estados):
        dim = 2**self.num_qubits
        ent = np.zeros(dim, dtype=complex)
        for i in range(min(dim, len(estados))):
            fase = np.exp(1j * np.pi * i / dim)
            ent[i] = estados[i % len(estados)] * fase
        ent /= np.linalg.norm(ent)
        self.matriz_densidad = np.outer(ent, np.conj(ent))
        return ent

    def medir_entrelazamiento(self):
        if self.matriz_densidad is None:
            return 0
        eigs = np.linalg.eigvalsh(self.matriz_densidad)
        eigs = eigs[eigs>1e-10]
        return -np.sum(eigs * np.log2(eigs))

    def propagacion_cuantica(self, entrada, puertas_cuanticas=None):
        estado = self.crear_superposicion(
            entrada.get('s_fti', []),
            entrada.get('s_fni', []),
            entrada.get('s_mfi', [])
        )
        for capa in range(self.num_capas):
            H = np.array([[1,1],[1,-1]])/np.sqrt(2)
            theta = np.pi*(capa+1)/self.num_capas
            Rx = np.array([
                [np.cos(theta/2), -1j*np.sin(theta/2)],
                [-1j*np.sin(theta/2), np.cos(theta/2)]
            ])
            # Aplicar Hadamard + Rx por pares
            temp = np.zeros_like(estado)
            for i in range(0, len(estado)-1, 2):
                bloque = estado[i:i+2]
                temp[i:i+2] = H@bloque
            for i in range(0, len(temp)-1, 2):
                bloque = temp[i:i+2]
                estado[i:i+2] = Rx@bloque
            if capa < self.num_capas-1:
                estado = self.crear_estado_entrelazado(estado)
        return estado

    def decoherencia_controlada(self, estado, tasa=0.1):
        ruido = (np.random.normal(0, tasa, estado.shape)
                + 1j*np.random.normal(0, tasa, estado.shape))
        estado = estado + ruido
        return estado/np.linalg.norm(estado)

    def interpretar_multiestado(self, estado_final):
        n = len(estado_final)
        out = []
        for i in range(n):
            amp = abs(estado_final[i])
            if amp>0.01:
                fase = np.angle(estado_final[i])
                prob = amp**2
                bin_str = format(i, f'0{int(math.log2(n))}b')
                out.append({
                    'estado': bin_str,
                    'amplitud': amp,
                    'fase': fase,
                    'probabilidad': prob,
                    'real': np.real(estado_final[i]),
                    'imag': np.imag(estado_final[i])
                })
        return sorted(out, key=lambda x: x['probabilidad'], reverse=True)

# 5. Analizador Híbrido: AnalizadorCuanticoHibrido
class AnalizadorCuanticoHibrido:
    def __init__(self, red_cuantica):
        self.red = red_cuantica

    def analisis_temporal(self, secuencia_temporal):
        resultados = []
        estados = []
        for t_step, datos in enumerate(secuencia_temporal):
            est = self.red.propagacion_cuantica(datos)
            est = self.red.decoherencia_controlada(est, 0.05*(t_step+1)/len(secuencia_temporal))
            entrel = self.red.medir_entrelazamiento()
            interp = self.red.interpretar_multiestado(est)
            resultados.append({
                'tiempo': t_step,
                'estado': est,
                'entrelazamiento': entrel,
                'interpretacion': interp,
                'coherencia': abs(np.vdot(est, est))
            })
            estados.append(est)
        corr = self.calcular_correlaciones_temporales(estados)
        return {
            'serie_temporal': resultados,
            'correlaciones': corr,
            'evolucion_entrelazamiento': [r['entrelazamiento'] for r in resultados]
        }

    def calcular_correlaciones_temporales(self, estados):
        n = len(estados)
        M = np.zeros((n,n), dtype=complex)
        for i in range(n):
            for j in range(n):
                M[i,j] = np.vdot(estados[i], estados[j])
        return M

    def visualizar_espacio_hilbert(self, estado):
        if len(estado)==2:
            rho = np.outer(estado, np.conj(estado))
            x = 2*np.real(rho[0,1])
            y = 2*np.imag(rho[0,1])
            z = np.real(rho[0,0]-rho[1,1])
            return {'x': x, 'y': y, 'z': z, 'radio': np.sqrt(x*x+y*y+z*z)}
        else:
            features = np.vstack([np.real(estado), np.imag(estado),
                                  np.abs(estado), np.angle(estado)]).T
            pca = PCA(n_components=3)
            proj = pca.fit_transform(features)
            return {'proyeccion_3d': proj, 'varianza_explicada': pca.explained_variance_ratio_}

# 6. Experimento Completo
def experimento_neuronal_cuantico():
    red = RedNeuronalCuantica(num_qubits=4, num_capas=3)
    analizador = AnalizadorCuanticoHibrido(red)

    # Generar secuencia de ejemplo
    seq = []
    for t_ in range(10):
        fase = 2*math.pi*t_/10
        seq.append({
            's_fti': [math.sin(fase), math.cos(fase), math.sin(2*fase)],
            's_fni': [math.cos(fase), math.sin(3*fase), math.cos(2*fase)],
            's_mfi': [math.sin(fase+math.pi/4), math.cos(fase-math.pi/3), 0.5]
        })

    res = analizador.analisis_temporal(seq)

    # Impresión resumida
    print("=== EXPERIMENTO CUÁNTICO ===")
    for paso in res['serie_temporal'][:3]:
        print(f"t={paso['tiempo']} | coherencia={paso['coherencia']:.3f} "
              f"| entrelaz={paso['entrelazamiento']:.3f}")
        for st in paso['interpretacion'][:2]:
            print(f"   |{st['estado']}> p={st['probabilidad']:.2f} fase={st['fase']:.2f}")
    return res

# 7. Entry point
if __name__ == '__main__':
    # 7.1. Ejemplo básico
    sistema = SistemaNeuroQuantico()
    basic = calcular_metricas_cuanticas(
        sistema,
        s_fti=[0.5,0.7,0.3],
        s_fni=[0.2,0.8,0.4],
        s_mfi=[0.6,0.9,0.1]
    )
    print("\n--- SISTEMA BASE ---")

    # Format the basic output as a table
    interp_data = basic['interpretacion']
    if interp_data['tipo'] == 'colapsado':
        table_data = {
            'Métrica': ['Tipo', 'Estado Clásico', 'Confianza', 'Incertidumbre Residual', 'Interpretación'],
            'Valor': [
                interp_data['tipo'],
                interp_data['estado_clasico'],
                f"{interp_data['confianza']:.3f}",
                f"{interp_data['incertidumbre_residual']:.3f}",
                interp_data['interpretacion']
            ]
        }
        df = pd.DataFrame(table_data)
        display(df)
    else:
        print(basic['interpretacion'])

    df.head
    # 7.2. Experimento avanzado
    experimento_neuronal_cuantico()

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import numpy as np

# Load the QASM circuit
qasm_code = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
h q[1];
h q[3];
ccx q[0], q[4], q[2];
cx q[0], q[1];
rx(pi/2) q[0];
rx(pi/2) q[1];
cx q[0], q[1];
h q[2];
cx q[3], q[4];
rz(pi/2) q[4];
cx q[3], q[4];
h q[0];
h q[4];
cx q[0], q[4];
swap q[0], q[4];
measure q[0] -> c[0];
"""

circuit = QuantumCircuit.from_qasm_str(qasm_code)

# Simulate the circuit
simulator = AerSimulator()

# Transpile the circuit for the simulator
compiled_circuit = transpile(circuit, simulator)

# Run the simulation
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()

# Get the measurement counts
counts = result.get_counts(compiled_circuit)
print("\nResultados de la simulación (mediciones):")
print(counts)

# Plotly visual resources
import plotly.graph_objects as go
from IPython.display import display
plotly = True
try:
    import plotly.io as pio
    pio.renderers.default = "iframe"
except ImportError:
    plotly = False

# You can also visualize the circuit
print("\nVisualización del circuito:")
display(circuit.draw(output='text'))
plt.Text(circuit.draw(output='text'))
plt.show()

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace
import numpy as np

def build_entangled_10qubit_circuit(circA: QuantumCircuit,
                                    circB: QuantumCircuit) -> QuantumCircuit:
    """
    Toma dos circuitos de 5 qubits (circA, circB),
    los une en 10 qubits y añade una capa de entrelazamiento cruzado.
    """
    # Creamos el circuito de 10 qubits
    circ10 = QuantumCircuit(10)
    # Componemos circA en los qubits [0..4], circB en [5..9]
    circ10.compose(circA, qubits=range(0,5), inplace=True)
    circ10.compose(circB, qubits=range(5,10), inplace=True)

    # Capa de entrelazamiento cruzado: CX(i -> i+5)
    for i in range(5):
        circ10.cx(i, i+5)
    return circ10

def extract_32x32_from_statevector(circ10: QuantumCircuit):
    """
    Simula el statevector, lo reordena en una matriz 32×32 y
    obtiene las densidades reducidas de cada bloque de 5 qubits.
    """
    # Simulamos el statevector
    sv = Statevector.from_instruction(circ10)
    amps = sv.data                      # Vector complejo de longitud 1024

    # 1) Matriz de amplitudes bipartita 32×32
    amp32 = amps.reshape((32, 32))      # filas = estados 5 qubits A, columnas = B

    # 2) Matriz de probabilidades
    prob32 = np.abs(amp32)**2           # cada elemento p(i,j)

    # 3) Matriz de densidad reducida de A (5 qubits)
    rhoA = partial_trace(sv, qargs=range(5,10)).data  # 32×32

    # 4) Matriz de densidad reducida de B (5 qubits)
    rhoB = partial_trace(sv, qargs=range(0,5)).data   # 32×32

    return {
        'amplitude_matrix': amp32,
        'probability_matrix': prob32,
        'rhoA': rhoA,
        'rhoB': rhoB
    }

# ----------------- EJEMPLO DE USO -----------------

# 1) Define dos “sub-circuitos neuronales” de 5 qubits
circA = QuantumCircuit(5)
# … aquí metes tus puertas para el módulo A …
circA.h(range(5))

circB = QuantumCircuit(5)
# … aquí metes tus puertas para el módulo B …
circB.rx(np.pi/4, range(5))

# 2) Construye el circuito global de 10 qubits y entrelázalo
circ10 = build_entangled_10qubit_circuit(circA, circB)

# 3) Extrae las matrices 32×32
data = extract_32x32_from_statevector(circ10)

print("Matriz de amplitudes 32×32:")
print(np.round(data['amplitude_matrix'], 3))

print("\nMatriz de probabilidades 32×32:")
print(np.round(data['probability_matrix'], 3))

print("\nRhoA (reducción en A):")
print(np.round(data['rhoA'], 3))

print("\nRhoB (reducción en B):")
print(np.round(data['rhoB'], 3))


In [ ]:
# 1. IMPORTS Y CONFIGURACIÓN (Combinados de todos los scripts)
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pylatexenc # Explicitly import pylatexenc

# Qiskit Imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import partial_trace, Statevector # Import Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer # Import circuit_drawer

# ==============================================================================
# 2. LÓGICA DEL SCRIPT 1: Pre-procesamiento e Interpretación
#    (Clase SistemaNeuroQuantico, ligeramente modificada para nuestro propósito)
# ==============================================================================

class CodificadorNeuroCuantico:
    """
    Adaptación del SistemaNeuroQuantico para actuar como un codificador clásico.
    Toma datos de entrada y los transforma en parámetros para el circuito cuántico.
    """
    def __init__(self, num_parametros):
        self.num_parametros = num_parametros
        # Parámetros globales del script original
        self.a_ft = 0.8
        self.a_mf = 0.9
        self.a_fn = complex(0, 1)
        self.peso_a, self.peso_b, self.peso_c, self.peso_d = 0.1, 0.3, 0.5, 0.7
        self.s, self.t, self.f = 0.5, 0.6, 0.001

    def crear_superposicion(self, s_fti, s_fni, s_mfi):
        norm_fti = np.array(s_fti) / (np.linalg.norm(s_fti) + 1e-9)
        norm_fni = np.array(s_fni) / (np.linalg.norm(s_fni) + 1e-9)
        norm_mfi = np.array(s_mfi) / (np.linalg.norm(s_mfi) + 1e-9)
        estado_base = np.array([
            self.a_ft * np.sum(norm_fti),
            self.a_fn * np.sum(norm_fni),
            self.a_mf * np.sum(norm_mfi)
        ], dtype=complex)
        return estado_base

    def funcion_onda(self, estado, tiempo):
        energia = np.array([self.f * i for i in range(1, len(estado) + 1)])
        fase = np.exp(-1j * energia * tiempo)
        return estado * fase

    def generar_parametros(self, s_fti, s_fni, s_mfi, a=1, b=0.5, c=0.7, d=0.3):
        """
        Genera un vector de parámetros (ángulos) para el PQC.
        """
        estado_super = self.crear_superposicion(s_fti, s_fni, s_mfi)
        influencia = np.array([a*self.peso_a, b*self.peso_b, c*self.peso_c, d*self.peso_d])
        psi_t = self.funcion_onda(estado_super, self.t)

        # Combinar todas las características en un vector complejo más grande
        features_combinadas = np.concatenate([psi_t, influencia, np.array([np.sum(influencia) * self.s])])

        # Generar los parámetros (ángulos) usando partes reales e imaginarias
        # y mapearlos al rango [0, 2*pi]
        params_real = np.real(features_combinadas)
        params_imag = np.imag(features_combinadas)

        long_vector = np.concatenate([params_real, params_imag])

        # Aseguramos que el vector tenga la longitud correcta repitiéndolo o truncándolo
        if len(long_vector) < self.num_parametros:
            long_vector = np.tile(long_vector, int(np.ceil(self.num_parametros / len(long_vector))))

        parametros_finales = long_vector[:self.num_parametros]

        # Normalizar y escalar para que sean ángulos
        parametros_finales = (parametros_finales - np.min(parametros_finales)) / (np.max(parametros_finales) - np.min(parametros_finales) + 1e-9)
        return parametros_finales * 2 * np.pi


# ==============================================================================
# 3. LÓGICA DE LOS SCRIPTS 2 Y 3: El procesador Cuántico
# ==============================================================================

class ProcesadorCuantico:
    """
    Construye y ejecuta el circuito cuántico parametrizado.
    """
    def __init__(self, num_qubits=10, num_capas=2):
        self.num_qubits = num_qubits
        self.num_neuronas_A = num_qubits // 2
        self.num_neuronas_B = num_qubits - self.num_neuronas_A
        self.num_capas = num_capas
        self.simulator = AerSimulator()

    def _capa_neuronal(self, num_qubits, prefijo):
        """Crea una capa de rotaciones parametrizadas para un sub-módulo."""
        qc = QuantumCircuit(num_qubits, name=f'Capa_{prefijo}')
        params_por_capa = num_qubits * 2

        # Asignar un parámetro a cada puerta de rotación
        from qiskit.circuit import Parameter
        parametros = [Parameter(f'{prefijo}_theta_{i}') for i in range(params_por_capa)]

        idx = 0
        for i in range(num_qubits):
            qc.ry(parametros[idx], i)
            idx += 1
            qc.rz(parametros[idx], i)
            idx += 1
        return qc, parametros

    def crear_pqc(self):
        """
        Crea el PQC completo de 10 qubits basado en la estructura del Script 3.
        """
        qc = QuantumCircuit(self.num_qubits)
        todos_parametros = []

        for capa in range(self.num_capas):
            # Módulo A (qubits 0 a 4)
            capa_A, params_A = self._capa_neuronal(self.num_neuronas_A, f'L{capa}A')
            qc.compose(capa_A, qubits=range(self.num_neuronas_A), inplace=True)
            todos_parametros.extend(params_A)

            # Módulo B (qubits 5 a 9)
            capa_B, params_B = self._capa_neuronal(self.num_neuronas_B, f'L{capa}B')
            qc.compose(capa_B, qubits=range(self.num_neuronas_A, self.num_qubits), inplace=True)
            todos_parametros.extend(params_B)

            # Capa de entrelazamiento (similar al Script 3)
            if capa < self.num_capas - 1: # No entrelazar después de la última capa
                qc.barrier()
                for i in range(self.num_neuronas_A):
                    qc.cx(i, i + self.num_neuronas_A)
                qc.barrier()

        self.pqc = qc
        self.parametros = todos_parametros
        return qc, todos_parametros

    def ejecutar_y_extraer(self, valores_parametros):
        """
        Asigna valores a los parámetros, ejecuta el circuito y extrae los resultados de la simulación.
        """
        # Crear el diccionario para asignar valores
        param_map = {param: val for param, val in zip(self.parametros, valores_parametros)}

        # Asignar los valores al circuito
        circuito_asignado = self.pqc.assign_parameters(param_map)

        # Añadir mediciones para obtener los conteos
        circuito_asignado.measure_all()

        # Run the circuit on the simulator
        job = self.simulator.run(circuito_asignado, shots=1024)
        result = job.result()

        # Get the measurement counts
        counts = result.get_counts(circuito_asignado)

        return {
            'counts': counts,
            'result_object': result # Return the full result object for potential further analysis
        }

# ==============================================================================
# 4. LA CLASE ORQUESTADORA: HybridQNN
# ==============================================================================

class HybridQNN:
    def __init__(self, num_qubits=10, num_capas=2):
        self.procesador_cuantico = ProcesadorCuantico(num_qubits, num_capas)

        # Construir el PQC y obtener el número de parámetros necesarios
        _, params = self.procesador_cuantico.crear_pqc()
        num_parametros_necesarios = len(params)
        print(f"QNN inicializada. El circuito cuántico requiere {num_parametros_necesarios} parámetros.")

        self.codificador_clasico = CodificadorNeuroCuantico(num_parametros_necesarios)

    def predict(self, input_data):
        """
        Ejecuta el pipeline completo: pre-procesamiento -> cuántico -> post-procesamiento.
        'input_data' debe ser un diccionario como {'s_fti': [...], 's_fni': [...], 's_mfi': [...]}
        """
        print("\n--- PASO 1: Pre-procesamiento Clásico ---")
        parametros_circuito = self.codificador_clasico.generar_parametros(**input_data)
        print(f"Generados {len(parametros_circuito)} ángulos para el PQC.")

        print("\n--- PASO 2: Ejecución de la Capa Cuántica ---")
        resultados_cuanticos = self.procesador_cuantico.ejecutar_y_extraer(parametros_circuito)
        print("Capa cuántica ejecutada. Resultados de mediciones obtenidos.")

        print("\n--- PASO 3: Post-procesamiento y Análisis ---")
        # En este ejemplo simplificado, el post-procesamiento es solo mostrar los conteos.
        # En una aplicación real, aquí se haría la clasificación o regresión.
        print("Resultados de las mediciones (conteos):")
        display(pd.DataFrame.from_dict(resultados_cuanticos['counts'], orient='index', columns=['Counts']))

        return resultados_cuanticos

    def visualizar_circuito(self):
        # Keep visualization if possible, but it might still fail due to pylatexenc
        display(self.procesador_cuantico.pqc.draw('text'))
        get_counts=True
        try:
            circuit_drawer(self.procesador_cuantico.pqc)
        except Exception as e:
            print(f"Circuit visualization failed: {e}")


# ==============================================================================
# 5. EJEMPLO DE USO
# ==============================================================================

# Inicializamos nuestra Red Neuronal Cuántica Híbrida
# 10 qubits en total (2 módulos de 5), 2 capas de procesamiento
qnn = HybridQNN(num_qubits=10, num_capas=2)

# Visualizamos la arquitectura del circuito cuántico que hemos creado
print("\nArquitectura del Circuito Cuántico Parametrizado (PQC):")
# This might still fail due to pylatexenc error, but the rest should run
try:
    qnn.visualizar_circuito()
except Exception as e:
    print(f"Circuit visualization failed: {e}")


# Generamos un dato de entrada de ejemplo (como en el Script 1)
input_sample = {
    's_fti': [math.sin(0.5), math.cos(0.5), math.sin(1.0)],
    's_fni': [math.cos(0.5), math.sin(1.5), math.cos(1.0)],
    's_mfi': [math.sin(0.5 + math.pi/4), math.cos(0.5 - math.pi/3), 0.5]
}

# Realizamos una "predicción"
resultados_finales = qnn.predict(input_sample)

In [ ]:
from matplotlib import pyplot as plt
df_1['Counts'].plot(kind='line', figsize=(8, 4), title='Counts')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# 5.1. Análisis del resultado y predicción
print("\n--- Análisis de Resultados y Predicción ---")
# Encontrar el estado con el mayor conteo
most_likely_state = max(resultados_finales['counts'], key=resultados_finales['counts'].get)
highest_count = resultados_finales['counts'][most_likely_state]

print(f"El estado vector con mayor probabilidad es '{most_likely_state}' con un conteo de {highest_count}.")
print(f"Este estado vector '{most_likely_state}' puede considerarse la predicción o respuesta más probable del QNN para esta entrada.")

# Puedes guardar este estado como tu "predicción"
prediction = most_likely_state